In [1]:
import os
os.chdir('C:/Users/theodore.wong/cif/cities-cif')
import ee, geemap, json
import geopandas as gpd
import rioxarray
#ee.Authenticate()
ee.Initialize()
from city_metrix.layers import Layer, OpenStreetMap, OpenStreetMapClass, WorldPop
from city_metrix.layers.layer import get_utm_zone_epsg

import warnings
warnings.filterwarnings('ignore')


FILEPATH = 'C:/Users/theodore.wong/danida_africa'

Could not find GEE credentials file, so prompting authentication.


In [2]:
boundary_filenames = os.listdir('{0}/boundaries'.format(FILEPATH))
boundary_filenames

['boundary-ETH-Dire_Dawa-ADM3.geojson',
 'boundary-ETH-Dire_Dawa-ADM3union.geojson',
 'boundary-KEN-Nairobi-ADM3.geojson',
 'boundary-KEN-Nairobi-ADM3union.geojson',
 'boundary-RWA-Kigali-ADM3.geojson',
 'boundary-RWA-Kigali-ADM4union.geojson',
 'boundary-RWA-Musanze-ADM2union.geojson',
 'boundary-RWA-Musanze-ADM3.geojson',
 'boundary-ZAF-CityOfJohannesburg-ADM3union.geojson',
 'boundary-ZAF-CityOfJohannesburg-ADM4.geojson',
 'boundary-ZAF-NelsonMandelaBay-ADM3union.geojson',
 'boundary-ZAF-NelsonMandelaBay-ADM4.geojson']

In [3]:
def buffer_gdf(gdf, buffer_distance_meters):
    source_crs = gdf.crs
    target_epsg = get_utm_zone_epsg(gdf.to_crs('EPSG:4326').total_bounds)
    gdf = gdf.to_crs(target_epsg)
    return gpd.GeoDataFrame({'geometry': gdf.buffer(buffer_distance_meters).to_crs('EPSG:4326')})

In [4]:
from geocube.api.core import make_geocube
def polygonsToRaster(gdf, outname):
    # https://spatial-dev.guru/2022/09/03/rasterize-vector-data-using-geopandas-and-geocube/
    
    gdf = gdf.to_crs('EPSG:4326')
    target_epsg = get_utm_zone_epsg(gdf.total_bounds)
    gdf = gdf.to_crs(target_epsg)
    target_gdf = gpd.GeoDataFrame({'geometry': gdf['geometry'], 'in_zone': 1})
    
    # Using GeoCube to rasterize the Vector
    raster = make_geocube(
        vector_data = target_gdf,
        measurements=["in_zone"],
        resolution=(-100, 100),
        output_crs = target_epsg,
        fill = 0
    )
    
    # Save raster census raster
    raster.rio.to_raster(outname)

In [50]:
class Isochrone(Layer):
    def __init__(self, filename, min_threshold=0, **kwargs):
        super().__init__(**kwargs)
        self.spatial_resolution = 100
        self.filename = filename
        self.min_threshold = min_threshold   # This is for number of nearby amenity locations
        
    def get_data(self, bbox):
        isochrone_raster = rioxarray.open_rasterio(self.filename)
        clipped = isochrone_raster.rio.reproject('EPSG:4326').rio.clip_box(*bbox, allow_one_dimensional_raster=True)
        return clipped[0].where(clipped[0] >= self.min_threshold)

In [6]:
fname = 'boundary-RWA-Kigali-ADM3.geojson'
cityname = '{0} {1}'.format(fname.split('-')[1], fname.split('-')[2])
print(cityname, end=' ')
with open('{0}/boundaries/{1}'.format(FILEPATH, fname), 'r') as ifile:
    json_obj = json.loads(ifile.read())
feature = geemap.geojson_to_ee(json_obj)
zones = geemap.ee_to_gdf(feature)

RWA Kigali 

In [13]:
schoolrastername = '{0}/schoolsraster_a_{1}.tiff'.format(FILEPATH, cityname.replace(' ', '-'))
schools = OpenStreetMap(osm_class=OpenStreetMapClass.SCHOOLS).get_data(zones.total_bounds)

# Create 500-meter proximity raster
polygonsToRaster(buffer_gdf(schools, 500), schoolrastername)

In [10]:
WorldPop().groupby(zones).sum()

C:\Users\theodore.wong\AppData\Local\anaconda3\envs\cif\lib\site-packages\xee\ext.py:685: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(


Extracting layer world pop from Google Earth Engine for bbox [29.97751933 -2.07980328 30.27715038 -1.77956142]:
[########################################] | 100% Completed | 836.97 ms


0      22734.281250
1      37880.578125
2      40248.343750
3      52910.382812
4      28968.460938
5      21364.787109
6      26939.201172
7      45500.085938
8      16139.497070
9      10387.324219
10     59446.511719
11     51281.859375
12     18179.992188
13     95782.187500
14     50661.371094
15     32298.679688
16     31584.472656
17     19513.593750
18     70253.484375
19    110299.132812
20     64630.542969
21     32906.179688
22     54711.843750
23     44438.742188
24     18786.589844
25     41704.695312
26     68044.000000
27     16742.509766
28     12788.495117
29     49337.164062
30     14939.575195
31     51926.351562
32     49360.179688
33     26229.808594
34     50603.242188
Name: sum, dtype: float64

In [52]:
#isochrone_layer = Isochrone(filename=schoolrastername)
WorldPop().mask(isochrone_layer).groupby(zones).sum()

C:\Users\theodore.wong\AppData\Local\anaconda3\envs\cif\lib\site-packages\xee\ext.py:685: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(


Extracting layer world pop from Google Earth Engine for bbox [29.97751933 -2.07980328 30.27715038 -1.77956142]:
[########################################] | 100% Completed | 720.29 ms


0      21335.462891
1        213.771103
2      52399.359375
3       4774.578125
4      20564.716797
5      24438.410156
6      33747.234375
7      16183.112305
8      10729.522461
9      58695.843750
10     39123.632812
11     18105.285156
12     96126.031250
13     50204.515625
14      6113.011719
15     31673.451172
16     19950.802734
17     69711.414062
18    109746.390625
19     64421.988281
20     33077.257812
21     54881.000000
22     41667.265625
23     16245.707031
24     16223.412109
25     67816.679688
26     16075.767578
27     12631.484375
28     48381.171875
29     15102.627930
30     52461.578125
31     38095.230469
32     26151.994141
33     50557.019531
Name: sum, dtype: float64

In [51]:
total_pop = WorldPop().groupby(zones).sum()
access_pop = WorldPop().mask(isochrone_layer).groupby(zones).sum()
access_fraction = access_pop / total_pop
zones['total_pop'] = total_pop
zones['access_pop'] = access_pop
zones['access_fraction'] = access_fraction
with open('{0}/schoolaccess_totalpopTEST-{1}.geojson'.format(FILEPATH, cityname.replace(' ', '-')), 'w') as ofile:
        ofile.write(zones.to_json())
print('(done)\n\n')

C:\Users\theodore.wong\AppData\Local\anaconda3\envs\cif\lib\site-packages\xee\ext.py:685: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(


Extracting layer world pop from Google Earth Engine for bbox [29.97751933 -2.07980328 30.27715038 -1.77956142]:
[########################################] | 100% Completed | 620.32 ms


C:\Users\theodore.wong\AppData\Local\anaconda3\envs\cif\lib\site-packages\xee\ext.py:685: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(


Extracting layer world pop from Google Earth Engine for bbox [29.97751933 -2.07980328 30.27715038 -1.77956142]:
[########################################] | 100% Completed | 524.27 ms
(done)




In [18]:
access_fraction

0     0.938471
1     0.005643
2     1.301901
3     0.090239
4     0.709900
5     1.143864
6     1.252718
7     0.355672
8     0.664799
9     5.650718
10    0.658132
11    0.353054
12    5.287463
13    0.524153
14    0.120664
15    0.980642
16    0.631665
17    3.572454
18    1.562149
19    0.584066
20    0.511790
21    1.667802
22    0.761577
23    0.365575
24    0.863563
25    1.626116
26    0.236255
27    0.754456
28    3.783179
29    0.306111
30    3.511584
31    0.733640
32    0.529820
33    1.927464
34         NaN
Name: sum, dtype: float64

In [72]:
worldpop_layer = WorldPop()
for fname in list(boundary_filenames):
    if fname.split('-')[3].split('.')[0][-1] != 'n':
        cityname = '{0} {1}'.format(fname.split('-')[1], fname.split('-')[2])
        print(cityname, end=' ')
        with open('{0}/boundaries/{1}'.format(FILEPATH, fname), 'r') as ifile:
            json_obj = json.loads(ifile.read())
        feature = geemap.geojson_to_ee(json_obj)
        zones = geemap.ee_to_gdf(feature)
        
        total_pop = WorldPop().groupby(zones).count() * WorldPop().groupby(zones).mean()
        zones['total_pop'] = total_pop
        
        schools = OpenStreetMap(osm_class=OpenStreetMapClass.SCHOOLS).get_data(zones.total_bounds)
        if len(schools) == 0:
            print('NO AMENITIES FOUND\n\n')
            zones['access_pop'] = -9999
            zones['access_fraction'] = -9999
            with open('{0}/schoolaccess_totalpop-{1}.geojson'.format(FILEPATH, cityname.replace(' ', '-')), 'w') as ofile:
                    ofile.write(zones.to_json())
        else:
            schoolrastername = '{0}/schoolsraster_a_{1}.tiff'.format(FILEPATH, cityname.replace(' ', '-'))
            buffered_4326 = gpd.GeoDataFrame({'geometry': buffer_gdf(schools, 500)})
            polygonsToRaster(buffered_4326, schoolrastername)

            isochrone_layer = Isochrone(filename=schoolrastername)
            total_pop = WorldPop().groupby(zones).count() * WorldPop().groupby(zones).mean()
            access_pop = WorldPop().mask(isochrone_layer).groupby(zones).count() * WorldPop().mask(isochrone_layer).groupby(zones).mean()

            zones['access_pop'] = access_pop
            zones['access_fraction'] = access_pop / total_pop
            with open('{0}/schoolaccess_totalpop_a-{1}.geojson'.format(FILEPATH, cityname.replace(' ', '-')), 'w') as ofile:
                    ofile.write(zones.to_json())
            print('(done)\n\n')

ETH Dire_Dawa Input covers too much area, splitting into 2 tiles


C:\Users\theodore.wong\AppData\Local\anaconda3\envs\cif\lib\site-packages\xee\ext.py:685: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(


Extracting layer world pop from Google Earth Engine for bbox [41.63315988  9.44538064 42.13315988  9.79335473]:
[########################################] | 100% Completed | 811.45 ms


C:\Users\theodore.wong\AppData\Local\anaconda3\envs\cif\lib\site-packages\xee\ext.py:685: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(


Extracting layer world pop from Google Earth Engine for bbox [42.13315988  9.57078713 42.42197622  9.84160302]:
[########################################] | 100% Completed | 705.93 ms
Input covers too much area, splitting into 2 tiles


...

C:\Users\theodore.wong\AppData\Local\anaconda3\envs\cif\lib\site-packages\xee\ext.py:685: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(


Extracting layer world pop from Google Earth Engine for bbox [ 25.19221 -34.05076  25.69221 -33.55308]:
[########################################] | 100% Completed | 1.26 sms


C:\Users\theodore.wong\AppData\Local\anaconda3\envs\cif\lib\site-packages\xee\ext.py:685: UserWarning: Unable to retrieve 'system:time_start' values from an ImageCollection due to: No 'system:time_start' values found in the 'ImageCollection'.
  warnings.warn(


Extracting layer world pop from Google Earth Engine for bbox [ 25.69221    -34.03228     25.868165   -33.60739586]:
[########################################] | 100% Completed | 581.44 ms


NoDataInBounds: No data found in bounds.